In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
cwd = os.getcwd()

In [3]:
raw_posts = pd.read_csv(cwd + '//AAPL Reddit Data//five-years-of-aapl-on-reddit-posts.csv')
raw_comments = pd.read_csv(cwd + '//AAPL Reddit Data//five-years-of-aapl-on-reddit-comments.csv')

##### Check no comments were erroneously added to the 'post' csv:

In [4]:
print(f"The raw_posts dataframe contains the types: {raw_posts['type'].unique()}")
print(f"The raw_comments dataframe contains the types: {raw_comments['type'].unique()}")

The raw_posts dataframe contains the types: ['post']
The raw_comments dataframe contains the types: ['comment']


#### All good.
##### A quick check of posts in nsfw subreddits tells us that the popularity of such posts is low (not exceeding a score of 11). For this reason, and that nsfw subreddits are unlikely to contain serious discussion of the AAPL ticker, they are dropped.

In [5]:
raw_posts[raw_posts['subreddit.nsfw'] == True][['subreddit.name', 'score']].max()

subreddit.name    worldpolitics
score                        11
dtype: object

In [6]:
raw_posts = raw_posts[raw_posts['subreddit.nsfw'] == False].copy()

##### Drop unneeded columns from each dataframe

In [7]:
raw_posts_comms = [raw_posts, raw_comments]
columns=['type', 'subreddit.id', 'subreddit.nsfw', 'permalink', 'domain', 'url']

def drop_columns(dfs, columns):
    for df in dfs:
        for col in columns:
            if col in list(df.columns):
                df.drop(columns=col, inplace=True)
            else:
                print('No columns could be dropped')

In [8]:
drop_columns(raw_posts_comms, columns)

No columns could be dropped
No columns could be dropped


##### Convert unix time values to datetime objects:

In [9]:
from datetime import datetime

In [10]:
def convert_datetime(dfs):
    for df in dfs:
        df['created_utc'] = df['created_utc'].apply(
            lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

        df.rename(columns={'created_utc': 'time_created'}, inplace=True)

        df['time_created'] = pd.to_datetime(df['time_created'])
        
def add_time_cols(dfs):
    for df in dfs:
        df['year_created'] = df['time_created'].dt.year
        df['month_created'] = df['time_created'].dt.month
        df['day_created'] = df['time_created'].dt.day
        
        df.rename(
            columns={'time_created': 'date_created',
                     'subreddit.name': 'subreddit',
                    }, inplace=True)
        
        df['hour_created'] = df['date_created'].dt.hour

In [ ]:
convert_datetime(raw_posts_comms)
add_time_cols(raw_posts_comms)

In [ ]:
clean_posts = raw_posts.copy()
clean_comms = raw_comments.copy()

In [ ]:
clean_posts.head()

In [ ]:
clean_comms.head()

##### Add column for the count of posts for a given subreddit

In [ ]:
clean_posts['sub_posts'] = clean_posts.groupby('subreddit')['id'].transform('count')

In [ ]:
pop_subs = clean_posts.groupby('subreddit')['sub_posts'].max().reset_index()
pop_subs = pop_subs.sort_values('sub_posts', ascending=False).set_index('subreddit')
pop_subs.head(10)

## <ins>Start of Post Analysis</ins>
### How do upvotes change over time?

In [ ]:
def create_plot(df, x_col, y_col, line=False, **kwargs):
    if line is True:
        for i, df in enumerate(df):
            plt.plot(df[x_col], df[y_col], label=labels[i], color=colors[i])
    
    elif line == False:
        for d in df:
            plt.bar(df[x_col], df[y_col], **kwargs)  

In [ ]:
monthly_post_scores = (
    clean_posts
        .groupby(['year_created', 'month_created'])['score']
        .mean()
        .reset_index()
    )
# month_mean_score

##### Quick check for high 2017 January mean

In [ ]:
# clean_posts[(clean_posts['month_created'] == 1) & (clean_posts['year_created'] == 2017)]
# clean_posts.sort_values(['score'], ascending=False).head()

#### Anomaly is high scoring post(s) in WSB reddit. I chose to filter out WSB posts due to 1) a much higher volume of posts 2) the top post(s) was/were 'baiting' upvotes (if you upvote, I'll...), and 3) most importantly to reduce bias, I will later perform analysis on posts from solely WSB, in combination with the non-WSB data. 

In [ ]:
WSB_posts = clean_posts[clean_posts['subreddit'] == 'wallstreetbets']
clean_posts = clean_posts[clean_posts['subreddit'] != 'wallstreetbets']
WSB_comments = clean_comms[clean_comms['subreddit'] == 'wallstreetbets']
clean_comms = clean_comms[clean_comms['subreddit'] != 'wallstreetbets']

In [ ]:
clean_comms.head()

##### Get dataframes with average and highest score for each month for each year.

In [ ]:
monthly_post_scores, monthly_comm_scores = clean_posts.copy(), clean_comms.copy()

for df in [monthly_post_scores, monthly_comm_scores]:
    df['year_month'] = df['date_created'].dt.strftime('%Y-%m')
    df['monthly_mean'] = df.groupby('year_month')['score'].transform('mean')
    df['monthly_max'] = df.groupby('year_month')['score'].transform('max')

In [ ]:
import matplotlib.dates as mdates

fig, ax = plt.subplots()
ax2=ax.twinx()

ax.plot(monthly_post_scores['year_month'], monthly_post_scores['monthly_mean'], color='k')
ax2.plot(monthly_post_scores['year_month'], monthly_post_scores['monthly_max'], color='tab:orange', linestyle='--')

ax.set_xlabel('Date')
ax.set_ylabel('Average Upvotes')
ax.set_title('Average Upvotes of $AAPL Related Posts on Reddit over Time')

ax.xaxis.set_major_locator(mdates.MonthLocator())
    
ax2.set_ylabel('Max Upvotes', color='tab:orange')

plt.show()

I'll later analyse this graph in combination with stock data, so no comments for now.

### Which subreddits have the highest average score?

In [ ]:
top_avg = (
    clean_posts
        .groupby('subreddit')['score']
        .mean()
        .reset_index()
        .sort_values('score', ascending=False)
        )

In [ ]:
create_plot(top_avg.head(20), 'subreddit', 'score', color='dimgrey', edgecolor='k')

plt.xticks(rotation='vertical')
plt.xlabel('Subreddit Name')
plt.ylabel('Average Score')
plt.title('Top 20 Subreddits with Highest Average Scores\n for $AAPL Related Posts')

plt.show()

### Which subreddits have the highest scoring posts?

In [ ]:
top_max = (
    clean_posts
        .groupby('subreddit')['score']
        .max()
        .reset_index()
        .sort_values('score', ascending=False)
        )

In [ ]:
create_plot(top_max.head(20), 'subreddit', 'score', color='dimgrey', edgecolor='k')

plt.xticks(rotation='vertical')
plt.xlabel('Subreddit Name')
plt.ylabel('Max Score')
plt.title('Top 20 Subreddits with Highest Scores\n for $AAPL Related Posts')

plt.show()

##### For some extra insight, I plot the prior two graphs again, but this time overlaid with each sub's max, or average score.

In [ ]:
avg_max_scores = pd.merge(left=top_avg, right=top_max, how='inner', left_on='subreddit', right_on='subreddit')
avg_max_scores.rename(columns={'score_x': 'average', 'score_y': 'max'}, inplace=True)

top_20_avg_sort = avg_max_scores.sort_values('average', ascending=False).head(20)
top_20_max_sort = avg_max_scores.sort_values('max', ascending=False).head(20)

In [ ]:
fig1, ax = plt.subplots()
ax1 = ax.twinx()

ax.bar(top_20_avg_sort['subreddit'], top_20_avg_sort['average'], color='dimgrey', edgecolor='k')
ax1.bar(top_20_avg_sort['subreddit'], top_20_avg_sort['max'], color='r', edgecolor='k', alpha=0.5)

ax.set_xlabel('Subreddit')
ax.set_ylabel('Average Score')
ax1.set_ylabel('Highest Score', color='r')

ax.set_title('Average and Highest Upvotes by 20 Highest Average Scoring\n$AAPL Related Posts on Reddit')

ax.tick_params('x', labelrotation=90)

plt.show()

In [ ]:
fig1, ax = plt.subplots()
ax1 = ax.twinx()

ax.bar(top_20_max_sort['subreddit'], top_20_max_sort['max'], color='dimgrey', edgecolor='k')
ax1.bar(top_20_max_sort['subreddit'], top_20_max_sort['average'], color='r', edgecolor='k', alpha=0.5)

ax.set_xlabel('Subreddit')
ax.set_ylabel('Highest Score')
ax1.set_ylabel('Average Score', color='r')

ax.set_title('Average and Highest Upvotes by 20 Highest Scoring\n$AAPL Related Posts on Reddit')

ax.tick_params('x', labelrotation=90)

plt.show()

No notable correlations here. 

### Do certain months receive higher scores?

Prior insights (simply looking at the most popular subreddits and their top scores, etc.) didn't warrant filtering the data too much, but we should now consider it given the results of those insights - the top scoring subs curtail after around 10 subs, and so taking a (separate) slice (as explained below) will better represent the majority of answers. 

In [ ]:
import numpy as np

plt.hist(clean_posts['score'], bins=[_ for _ in np.arange(0, 100, 1)])
plt.show()

In [ ]:
print(f"The first 71% of values are equal to or less than {clean_posts['score'].quantile(0.71)}")
print(f"The first 72% of values are equal to or less than {clean_posts['score'].quantile(0.72)}")

The vast majority (72%) of values are less than or equal to 1.

I therefore create a new series for top posts, just to get an idea of trends, while keeping the clean_posts data as itself. This allows me to more easily compare the top posts to the whole dataset, where the axes is not terribly skewed by the (very) top values. This series contains values between the top 10% and top 1% of values.

In [ ]:
pctl_90 = clean_posts['score'].quantile(0.9)
pctl_99 = clean_posts['score'].quantile(0.99)

top_posts = clean_posts[(clean_posts['score'] > pctl_90) & (clean_posts['score'] > pctl_90)]
month_scores = top_posts.groupby(['month_created', 'year_created'])['score'].mean().reset_index()

print(f'90th Percentile: {pctl_90}')
print(f'99th Percentile: {pctl_99}')

In [ ]:
plt.hist(top_posts['score'], bins=[_ for _ in np.arange(0, 190, 1)])
plt.show()

Data is still positively skewed, but notably less so.

#### From hereon, posts with scores which lie between the 90th and 99th percentile will be referred to as 'top posts' in commentary. 

In [ ]:
import time
from IPython import display      

colors = {2017:'#3bb410', 2018:'#e9da10', 2019:'#df951e', 2020:'#d2622f', 2021:'#c44141'}
years = [2017, 2018, 2019, 2020]

plt.xticks(month_scores['month_created'])
plt.xlabel('Month')
plt.ylabel('Average Score')
plt.yticks(range(0, 1000, 100))
plt.title('Average of Scores for Top Scorers, by Month')

for year in years:
    plt.plot(
        month_scores[month_scores['year_created'] == year]['month_created'],
        month_scores[month_scores['year_created'] == year]['score'],
        color=colors[year],
        label=year)
          
    plt.legend(loc='upper left')
        
    display.display(plt.gcf())
    display.clear_output(wait=True)
    time.sleep(1.0)
    
plt.show()

#### Re-run this cell to animate!

Average score seems to spike around the middle of the year, notably so between May and August. Analysing $AAPL stock data for each month will be interesting.

### What about weekday?

In [ ]:
all_weekday = clean_posts.copy()
top_weekday = top_posts.copy()

for df in [all_weekday, top_weekday]:
    df['weekday'] = df['date_created'].dt.strftime('%A') 

cats = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

all_weekday['all_weekday_mean'] = all_weekday.groupby('weekday')['score'].transform('mean')
all_weekday['all_weekday_count'] = all_weekday.groupby('weekday')['id'].transform('count')

all_weekday['top_weekday_mean'] = top_weekday.groupby('weekday')['score'].transform('mean')
all_weekday['top_weekday_count'] = top_weekday.groupby('weekday')['id'].transform('count')

weekday_posts = (all_weekday.groupby('weekday').mean().reindex(cats))

In [ ]:
all_weekday

In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()

ax.plot(weekday_posts.index, weekday_posts['all_weekday_count'], color='k')
ax2.plot(weekday_posts.index, weekday_posts['all_weekday_mean'], color='tab:orange')

ax.set_xlabel('Weekday')
ax.set_ylabel('Number of Posts')
ax2.set_ylabel('Average Score', color='tab:orange')

ax.set_title('Number and Average Score of All Posts by Weekday')

plt.show()

As the week progresses, the number of posts stays consistent until the weekend. This is expected; market is closed on the weekend (thus no concurrent activity to discuss). The average score of posts decreaes across the week, and rises again on Monday. Potential reasons could include excitement for stock market re-open, people not wanting to hold short term positions over the weekend and therefore not interacting with posts, or people deciding/discussing their positions at the start of the week. 

In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()

ax.plot(weekday_posts.index, weekday_posts['top_weekday_count'], color='k')
ax2.plot(weekday_posts.index, weekday_posts['top_weekday_mean'], color='tab:orange')

ax.set_xlabel('Weekday')
ax.set_ylabel('Number of Posts')
ax2.set_ylabel('Average Score', color='tab:orange')

ax.set_title('Number and Average Score of Top Posts by Weekday')

plt.show()

The top posts follow a very similar trend.

### How do the score and count of posts vary by time of day?

In [ ]:
all_hour_count = clean_posts.groupby('hour_created')['id'].count().reset_index()
all_hour_avg_score = clean_posts.groupby('hour_created')['score'].mean().reset_index()

top_hour_avg_score = top_posts.groupby('hour_created')['score'].mean().reset_index()
top_hour_avg_count = top_posts.groupby('hour_created')['id'].count().reset_index()

##### Set larger fig size

In [ ]:
plt.rcParams["figure.figsize"] = (10, 7)

In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()

ax.plot(
    all_hour_count['hour_created'],
    all_hour_count['id'],
    color='k',
    label='Posts/Hour'
)
ax2.plot(
    top_hour_avg_score['hour_created'],
    top_hour_avg_score['score'],
    label='Avg Score/Hour (90th-99th PCTL)',
    linestyle='dotted',
    color='r'
)
ax2.plot(
    all_hour_avg_score['hour_created'],
    all_hour_avg_score['score'],
    label='Avg Score/Hour all Posts',
    color='tab:orange'
)

ax.set_xticks(all_hour_count['hour_created'])
ax.set_xlabel('Hour (UTC)')
ax.set_ylabel('Number of Posts')
ax.set_title('Number of $AAPL-related Posts by Hour\nand their Mean Score')

ax2.set_ylabel('Average Score', color='r')

handles, labels = ax.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()

ax2.legend(handles + handles2, labels + labels2, loc='upper left')

ax.grid()
plt.show()

To a degree, it seems that the highest scoring posts lag behind when the number of posts is highest. That is, posting between the hours of 9am and 12pm, before the post-quantity upturn between 11am and 4pm, results in the highest scoring $AAPL related posts. Most notably, the quantity of posts begins to steeply rise at NASDAQ pre-market open and LSE market-open (9am UTC), with number of posts beginning to reach its peak at around NASDAQ market open (2:30pm UTC). AAPL-related posts briefly decrease in quantity at 7pm UTC, perhaps a result of dinner times in the United Kingdom, lunch times in the US, or stable market conditions prior to end-of-day trading. Post-quantity sharply decreases following market-close (9pm UTC) until pre-market open.

Given that post-score rises in combination with pre-market open, it could be hypothesised that due-diligence posts are at lease partially responsible; discussion/reasoning around market-plays in this period is valuable towards deciding the positions one will take at market open. In tandem, at market close, post-score average takes a turn updwards for the highest scoring posts (and to a lesser degree, for all posts); people are discussing the market conditions for the day, and the positions they will take on the following day. Let's explore that hypothesis.

#### Sets figsize back to normal for all subsequent plots

In [ ]:
plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

In [ ]:
all_DD, top_DD, wsb_DD = clean_posts.copy(), top_posts.copy(), WSB_posts.copy()

dataframes = [
    df[(df['title'].str.contains('DD|diligence|due diligence')==True) 
       | (df['selftext'].str.contains('DD|diligence|due diligence')==True)
      ] for df in [all_DD, top_DD, wsb_DD]
    ]

mean_dfs = [df.groupby('hour_created')['score'].mean().reset_index() for df in dataframes]
count_dfs = [df.groupby('hour_created')['id'].count().reset_index() for df in dataframes]

In [ ]:
labels = {0: 'All DD Posts', 1: 'Top DD Posts', 2: 'WSB DD Posts'}
colors = {0: 'tab:red', 1: 'tab:orange', 2: 'tab:blue'}   

create_plot([df for df in count_dfs], 'hour_created', 'id', line=True, label=labels, color=colors)
        
plt.xticks(all_hour_count['hour_created'])
plt.xlabel('Hour (UTC)')
plt.ylabel('Number of Posts')
plt.title('Number of Due-Diligence Related Posts by Hour')

plt.legend()
plt.show()

DD for all posts (non-WSB) posts increases dramatically at pre-market open (9am UTC). This is expected, given that people are deciding/discussing their positions prior to the trading period. At NASDAQ market open (2:30 UTC), post-volume begins to decrease, and then rises somewhat after market close until midnight UTC. 

A higher number of Top DD and WSB DD posts are made later in the day (towards market close); for WSB, this might reflect something as little as a US-centric user base, or as much as more sporadic trading behaviour (i.e. more focus on day-trading). 

In [ ]:
create_plot([df for df in mean_dfs], 'hour_created', 'score', line=True, labels=labels, colors=colors)

plt.xticks(all_hour_count['hour_created'])
plt.xlabel('Hour (UTC)')
plt.ylabel('Average Score')
plt.title('Average Score of Due-Diligence Related Posts by Hour')

plt.legend()
plt.show()

While there is a relatively little number of posts with 'DD' or 'diligence' mentioned, such posts do exercise leverage during premarket hours on those where DD is not mentioned (see 'Number of $AAPL-related Posts by Hour and their Mean Score' plot). See, in particular, the higher average scores for top-DD posts at 4am UTC (end of after-market). WSB are upvoted throughout trading hours when post-quantity is high, but receive a spike in upvotes after market close, despite a decline in post-quantity. Therefore, posting DD to WSB at 11pm UTC will result in the highest likelihood of having a highly upvoted post (a drop in post-quantity but a rise in average score).

I won't be exploring this further with statistical testing, as the analysis was just for practice/fun, and analysis of reddit data was really meant to be preliminary. 

## <ins>Start of Comment Analysis</ins>

In [ ]:
monthly_comm_scores.head()

In [ ]:
monthly_comm_scores['ymd'] = monthly_comm_scores['date_created'].dt.strftime('%Y-%m-%d')

In [ ]:
sentiment = (
    monthly_comm_scores[clean_comms['sentiment'].notna()]
    .groupby('ymd')
    .mean()
    .reset_index()
    )

sentiment_mean = sentiment.groupby('year_created')['sentiment'].mean().reset_index()
sentiment.head()

In [ ]:
from matplotlib.colors import BoundaryNorm

#This cell uses code from: https://stackoverflow.com/questions/23994020/colorplot-that-distinguishes-between-positive-and-negative-values

fig, ax = plt.subplots()
  
cmap = plt.get_cmap('RdYlGn')
cmaplist = [cmap(i) for i in range(cmap.N)]
cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)

# define the bins and normalize and forcing 0 to be part of the colorbar.
bounds = np.arange(np.min(-0.4),np.max(0.4),.03)
idx=np.searchsorted(bounds,0)
bounds=np.insert(bounds,idx,0)
norm = BoundaryNorm(bounds, cmap.N)
    
ax.scatter(sentiment['ymd'],
           sentiment['sentiment'],
           c=sentiment['sentiment'], norm=norm, cmap=cmap)

ax.set_xlabel('Date')
ax.set_ylabel('Sentiment')
ax.set_yticks(np.arange(-0.4, 0.8, 0.2))
ax.set_title('Daily Average Sentiment Distribution\nfor $AAPL-related Comments by Month and Year')

ax.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, interval=12))
plt.show()

In [ ]:
plt.scatter(
    sentiment['year_created'],
    sentiment['sentiment'],
    c=sentiment['sentiment'], norm=norm, cmap=cmap)

plt.scatter(
    sentiment_mean['year_created'],
    sentiment_mean['sentiment'],
    color='lightblue', edgecolor='k',
    label='Mean')

plt.legend()

plt.xlabel('Date')
plt.ylabel('Sentiment')
plt.title('Daily Average Sentiment Distribution\nfor $AAPL-related Comments by Year')

plt.show()

In [ ]:
plt.scatter(sentiment['sentiment'], sentiment['score'], c=sentiment['sentiment'], norm=norm, cmap=cmap)

plt.xlabel('Sentiment')
plt.ylabel('Score')
plt.title('Daily Average Sentiment by Score\nDistribution for $AAPL-related Comments')

plt.show()

## <ins>Start of Stock Analysis</ins>

### Use mplfinance for candlesticks
#### How does reddit activity change with stock price?
#### How do upvotes change with stock price?

In [ ]:
import mplfinance as mpf

raw_stock_data = pd.read_csv(cwd + '//AAPL Stock Data//AAPL-Stock-Data.csv', index_col='Date', parse_dates=True)
raw_stock_data.isnull().values.any()

In [ ]:
raw_stock_data.head()

#### Unfortunately, we only have date values (no time data). Analysis will therefore only be conducted on post/comment values by day. 

##### We need to get date values beginning from November 1st 2016, the starting date for posts and comments.

In [ ]:
raw_stock_data = raw_stock_data[raw_stock_data.index >= '2016-11-01']

In [ ]:
raw_stock_data
weekly_stock_data = raw_stock_data.resample('W-Mon').mean()
monthly_stock_data = raw_stock_data.resample('M').mean()

monthly_stock_data.head()

In [ ]:
sentiment_monthly = sentiment.groupby(['year_created', 'month_created']).mean().reset_index()
sentiment_monthly.head()

In [ ]:
apds = [mpf.make_addplot(sentiment_monthly['sentiment'],
                         width=1,
                         color='tab:blue',
                         ylabel='Average Sentiment'),
        mpf.make_addplot(sentiment_monthly['score'],
                         width=1,
                         color='tab:orange',
                         panel=1,
                         ylabel='Averge Score', linestyle='--')
       ]

s = mpf.make_mpf_style(base_mpf_style='yahoo', y_on_right=False)

mpf.plot(monthly_stock_data, type='candlestick', style=s, addplot=apds, figscale=1, ylabel='Price (USD)')